# Lab1

### Imports 

In [133]:
import os
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import xml.etree.ElementTree as ET 
import pycrfsuite

### Variables

In [134]:
inputdir = "data/Devel"
drug_bank_path = "resources/DrugBank.txt"
stopwords = set(stopwords.words('english')) 

### Functions

In [135]:
def extract_drug_bank(drug_bank_path):
    drug_bank = {}
    with open(drug_bank_path, encoding="utf8") as f:
        for line in f:
            data = line.strip().split('|')
            drug_name = data[0]
            drug_type = data[1]
            drug_bank[drug_name.lower()] = drug_type
    return drug_bank

drug_bank = extract_drug_bank(drug_bank_path)

In [140]:
def tokenize(text): 
    list_tokens = []
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords]
    ini_token = 0
    for token in tokens:
        ini_token = text.find(token, ini_token)
        list_tokens.append((token, ini_token, ini_token+len(token)-1))
        ini_token += len(token)
        
    return list_tokens

def extract_features(tokens):
    features = []
    
    for i in range(len(tokens)):
        token = tokens[i]
        word = token[0]
        
        feature_vector = [
            'bias',
            'word.lower=' + word.lower(),
            'word[-3:]=' + word[-3:],
            'word[-2:]=' + word[-2:],
            'word.isupper=%s' % word.isupper(),
            'word.istitle=%s' % word.istitle(),
            'word.isdigit=%s' % word.isdigit()
        ]
        if i > 0:
            word1 = tokens[i-1][0]
            feature_vector.extend([
                '-1:word.lower=' + word1.lower(),
                '-1:word.istitle=%s' % word1.istitle(),
                '-1:word.isupper=%s' % word1.isupper()
            ])
        else:
            feature_vector.append('BOS')

        if i < len(sent)-1:
            word1 = sent[i+1][0]
            feature_vector.extend([
                '+1:word.lower=' + word1.lower(),
                '+1:word.istitle=%s' % word1.istitle(),
                '+1:word.isupper=%s' % word1.isupper()
            ])
        else:
            feature_vector.append('EOS')
            
        features.append(feature_vector)
        
    return features

def output_features(sent_id, tokens, entities, features):
    for i in range(len(tokens)):
        
    
    return

def output_entities(sent_id, tokens, classes, outf):
    i = 0
    while i < len(tokens):
        token = tokens[i][0]
        offset_ini = tokens[i][1]
        offset_end = tokens[i][2]
        token_BIO = classes[i][0]
        if token_BIO == 'B':
            token_class = classes[i][2:]
        ++i
        
        if token_BIO == 'B':
            while i < len(tokens) and classes[i].startsWith('I'):
                token = token + ' ' + tokens[i][0]
                offset_end = tokens[i][2]
                ++i
                
            outf.write(sent_id+'|'+str(offset_ini)+'-'+str(offset_end)+'|'+token+'|'+token_class)
            outf.write("\n")
    return

def evaluate(inputdir, outputfile):
    os.system("java -jar eval/evaluateNER.jar "+ str(inputdir) + " " + str(outputfile))
    return

def my_nerc(inputdir, outputfile):
    outf = open(outputfile, "w")
    # Read files and parse files
    for filename in os.listdir(inputdir):     
        fullname = os.path.join(inputdir, filename)
        tree = ET.parse(fullname)
        root = tree.getroot()    
        
        for sentence in root.findall('sentence'):
            # Get sentence id and tokenize text
            sent_id = sentence.get('id')
            tokens = tokenize(sentence.get('text'))
            entities = extract_entities(tokens)
            output_entities(sent_id, entities, outf)
    
    outf.close()
    evaluate(inputdir, outputfile)
    return

In [141]:
my_nerc(inputdir, 'task9.1_devel_1.txt')
